In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import sqlite3 as db
import sys
import json
print(sys.version)

3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]


Справочники

In [2]:
members_df = pd.read_csv('data/Справочник участников оборота товаров.csv')

points_df = pd.read_csv('data/Справочник торговых точек.csv')

points_df.loc[points_df['region_code'] == 51, 'city_with_type'] = "Мурманская"
points_df.loc[points_df['region_code'] == 77, 'city_with_type'] = "Москва"
points_df.loc[points_df['region_code'] == 50, 'city_with_type'] = "Мо"
points_df.loc[points_df['region_code'] == 78, 'city_with_type'] = "Спб"
points_df['postal_code'] = points_df['postal_code'].fillna(0).astype(int)

products_df = pd.read_csv('data/Справочник продукции.csv')

products_df['country'] = products_df['country'].fillna('0')
products_df['volume'] = products_df['volume'].fillna(1)
products_df.loc[products_df['volume'] == "НЕ КЛАССИФИЦИРОВАНО", 'volume'] = 1

In [3]:
inn_points_region_code_equal = points_df.set_index('inn').join(members_df.set_index('inn')['region_code'], rsuffix='_inn').query("region_code == region_code_inn").index.unique()

inn_points_region_code_NOT_equal = set(members_df['inn'].unique()) - set(inn_points_region_code_equal)
inn_points_region_code_NOT_equal = np.array(list(inn_points_region_code_NOT_equal))
inn_points_region_code_NOT_equal.size

11274

Агрегированные

In [4]:
movement_df = pd.read_csv('data/Агрегированные данные о перемещениях товаров между участниками с 2021-11-22 по 2022-11-21.csv')
out_df = pd.read_csv('data/Агрегированные данные о выводе товаров из оборота с 2021-11-22 по 2022-11-21.csv')
in_df = pd.read_csv('data/Агрегированные данные о вводе товаров в оборот с 2021-11-22 по 2022-11-21.csv')

Производитель

In [5]:
factory_in = pd.read_csv('data/Дополнительные датасеты для продукта для производителя/Данные о вводе товаров в оборот с 2021-11-22 по 2022-11-21 один производитель.csv')
factory_out = pd.read_csv('data/Дополнительные датасеты для продукта для производителя/Данные о выводе товаров из оборота с 2021-11-22 по 2022-11-21 один производитель.csv')
factory_move = pd.read_csv('data/Дополнительные датасеты для продукта для производителя/Данные о перемещениях товаров между участниками с 2021-11-22 по 2022-11-21 один производитель.csv')

Фильтрация 

Оставляем только продажи конечному

In [6]:
factory_out = factory_out.query('type_operation == "Продажа конечному потребителю в точке продаж"')

Джойненные

In [7]:
factory_out_joined = pd.read_csv('data_joined/factory_out_joined.csv')
factory_move_joined = pd.read_csv('data_joined/factory_move_joined.csv')

In [8]:
factory_out_joined['date'] = pd.to_datetime(factory_out_joined['date'])
factory_move_joined['date'] = pd.to_datetime(factory_move_joined['date'])

factory_out_joined['quarter'] = factory_out_joined['date'].dt.quarter
factory_move_joined['quarter'] = factory_move_joined['date'].dt.quarter

Данные с запроса

In [9]:
#  9199AB529CF62D4BDB7E8B1D7459001D
# 2868B842B46F754C088D89E130F89585
# C6AD4B84092CCBB3E3729F73B00C55A1
# 75E0EAF2E5062A080CA0FDBBB8140627
# 2169F5521C7AE342374A8298DBCA26FD
#MY_SELECTED_PRODUCT = "9199AB529CF62D4BDB7E8B1D7459001D" 

In [10]:
PRODUCT_SHORT = "9199AB529CF62D4BDB7E8B1D7459001D"
REGION_CODE = 77
POSTAL_CODE = 0 
TIME_H = 0 #
GROUP_PERIOD =  'week' # 'dayy' 'month' 'quarter'

Перемещения

In [11]:
one_prod_region = factory_move_joined.query(f"product_short_name == '{PRODUCT_SHORT}'").query(f"region_code_rec_inn == {REGION_CODE}")

In [12]:
# Фильтрация ИНН на критерий нахождения в points_df
receiver_inns = factory_move_joined.query(f"product_short_name == '{PRODUCT_SHORT}'").query(f"region_code_rec_inn == {REGION_CODE}")['receiver_inn'].unique()
list_points_inn = set(receiver_inns).intersection(points_df['inn'].unique())

In [13]:
# factory_move_joined.query("product_short_name == '9199AB529CF62D4BDB7E8B1D7459001D'").query(f"region_code_rec_inn == {REGION_CODE}").groupby(['receiver_inn']).count()['sender_inn'].sort_values(ascending=False)

one_prod_region_filtered = one_prod_region[one_prod_region['receiver_inn'].isin(list_points_inn)]
one_prod_region_filtered = one_prod_region_filtered.query("receiver_inn != 'DA62EC79660CF21AC37A260DA6F642C4'")

one_prod_region_filtered_moving = one_prod_region_filtered.groupby([GROUP_PERIOD])['cnt_moved'].sum() # Группировка
one_prod_region_filtered_moving.head(2)

week
1    475
2    842
Name: cnt_moved, dtype: int64

Выбытия

In [14]:
one_good_type_region = factory_out_joined.query(f"product_short_name == '{PRODUCT_SHORT}'").query(f"region_code == {REGION_CODE}")

one_good_type_region_sum = one_good_type_region.groupby(['week'])['cnt'].sum() # Группировка
one_good_type_region_sum.head(2)

week
1    1361
2     550
Name: cnt, dtype: int64

In [15]:
factory_move_joined.shape

(342302, 19)

Джойним чтобы достать postal code

In [25]:
factory_move_joined.set_index('receiver_inn').join(points_df.drop_duplicates().set_index('inn')['postal_code'], how='inner')\ # consider left
.reset_index().rename(columns={'index':'receiver_inn'}).drop(columns=['Unnamed: 0'])\
.drop_duplicates(subset=list(factory_move_joined.columns)[1:])

,receiver_inn,sender_inn,gtin,dt,prid,cnt_moved,product_name,product_short_name,brand,country,region_code,region_code_rec_inn,date,month,week,day,dayy,quarter,postal_code
0,001302A80A24238AFF2C225ED859BE24,4D335F39E9F496F935C0397D78D57285,06F4CAE6A2764D141E53C3AC8282CD36,2022-07-04,DA62EC79660CF21AC37A260DA6F642C4,10,3450999CEDBFE595CD0DDE4A287CF500,C6AD4B84092CCBB3E3729F73B00C55A1,550710FDA7CDAABA81268E683325B534,РОССИЯ,22,54.0,2022-07-04,7,27,4,185,3,632331.0
1,001302A80A24238AFF2C225ED859BE24,8C6B3D74EF25A874F391B8764C0882AA,06F4CAE6A2764D141E53C3AC8282CD36,2022-10-18,DA62EC79660CF21AC37A260DA6F642C4,6,3450999CEDBFE595CD0DDE4A287CF500,C6AD4B84092CCBB3E3729F73B00C55A1,550710FDA7CDAABA81268E683325B534,РОССИЯ,54,54.0,2022-10-18,10,42,18,291,4,632331.0
2,001302A80A24238AFF2C225ED859BE24,8C6B3D74EF25A874F391B8764C0882AA,06F4CAE6A2764D141E53C3AC8282CD36,2022-11-01,DA62EC79660CF21AC37A260DA6F642C4,10,3450999CEDBFE595CD0DDE4A287CF500,C6AD4B84092CCBB3E3729F73B00C55A1,550710FDA7CDAABA81268E683325B534,РОССИЯ,54,54.0,2022-11-01,11,44,1,305,4,632331.0
3,001302A80A24238AFF2C225ED859BE24,8C6B3D74EF25A874F391B8764C0882AA,06F4CAE6A2764D141E53C3AC8282CD36,2022-07-26,DA62EC79660CF21AC37A260DA6F642C4,4,3450999CEDBFE595CD0DDE4A287CF500,C6AD4B84092CCBB3E3729F73B00C55A1,550710FDA7CDAABA81268E683325B534,РОССИЯ,54,54.0,2022-07-26,7,30,26,207,3,632331.0
4,001302A80A24238AFF2C225ED859BE24,8C6B3D74EF25A874F391B8764C0882AA,06F4CAE6A2764D141E53C3AC8282CD36,2022-07-12,DA62EC79660CF21AC37A260DA6F642C4,5,3450999CEDBFE595CD0DDE4A287CF500,C6AD4B84092CCBB3E3729F73B00C55A1,550710FDA7CDAABA81268E683325B534,РОССИЯ,54,54.0,2022-07-12,7,28,12,193,3,632331.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7314228,FFF9E9E005A8F64CB3F927D76CCBE34B,AE7F5C67298A67CACD2A193C9405524F,C4F2899DD2A62E0EAE45F93019438C05,2021-11-25,DA62EC79660CF21AC37A260DA6F642C4,1,E736D2D4470658513B56DED9412A29EC,2868B842B46F754C088D89E130F89585,550710FDA7CDAABA81268E683325B534,РОССИЯ,50,76.0,2021-11-25,11,47,25,329,4,150010.0
7314230,FFF9E9E005A8F64CB3F927D76CCBE34B,AE7F5C67298A67CACD2A193C9405524F,C4F2899DD2A62E0EAE45F93019438C05,2022-07-28,DA62EC79660CF21AC37A260DA6F642C4,1,E736D2D4470658513B56DED9412A29EC,2868B842B46F754C088D89E130F89585,550710FDA7CDAABA81268E683325B534,РОССИЯ,50,76.0,2022-07-28,7,30,28,209,3,150010.0
7314232,FFF9E9E005A8F64CB3F927D76CCBE34B,AE7F5C67298A67CACD2A193C9405524F,D3D15D7601B1BC10960C17B61C190353,2021-11-25,DA62EC79660CF21AC37A260DA6F642C4,2,524574517B4F0E01D0905D34DDDA3F3C,C6AD4B84092CCBB3E3729F73B00C55A1,550710FDA7CDAABA81268E683325B534,РОССИЯ,50,76.0,2021-11-25,11,47,25,329,4,150010.0
7314234,FFF9E9E005A8F64CB3F927D76CCBE34B,AE7F5C67298A67CACD2A193C9405524F,D3D15D7601B1BC10960C17B61C190353,2022-04-20,DA62EC79660CF21AC37A260DA6F642C4,2,524574517B4F0E01D0905D34DDDA3F3C,C6AD4B84092CCBB3E3729F73B00C55A1,550710FDA7CDAABA81268E683325B534,РОССИЯ,50,76.0,2022-04-20,4,16,20,110,2,150010.0


Считаем

In [17]:
# one_prod_region_filtered_moving - one_good_type_region_sum

In [26]:
# cumsum
df_total = pd.DataFrame({'move': one_prod_region_filtered_moving, 'out': one_good_type_region_sum})
df_total['move_out'] = df_total['move'] - df_total['out']

df_total['move_cumsum'] = df_total['move'].cumsum()
df_total['out_cumsum'] = df_total['out'].cumsum()
df_total['cum_sum_diff'] = df_total['move_cumsum'] - df_total['out_cumsum']

In [27]:
df_total

,move,out,move_out,move_cumsum,out_cumsum,cum_sum_diff
week,,,,,,
1,475,1361,-886,475,1361,-886
2,842,550,292,1317,1911,-594
3,1112,527,585,2429,2438,-9
4,961,424,537,3390,2862,528
5,618,502,116,4008,3364,644
6,766,597,169,4774,3961,813
7,1117,546,571,5891,4507,1384
8,1470,697,773,7361,5204,2157
9,1921,1085,836,9282,6289,2993


In [20]:
# pd.date_range('2021-11-22', '2022-11-21', freq='D').week

# import datetime
# d = f"{2022}-W{6}"
# r = datetime.datetime.strptime(d + '-1', "%Y-W%W-%w")
# print(str(r).split(' ')[0])

In [21]:
week_to_dt_pd = factory_move_joined.sort_values(by='dt').groupby(['week']).first().sort_values(by='dt')['dt'].reset_index()

In [22]:
week_nums = np.unique(pd.date_range('2021-11-22', '2022-11-21', freq='D').week.values)

C:\Users\Dima\AppData\Local\Temp\ipykernel_12308\3114958978.py:1: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  week_nums = np.unique(pd.date_range('2021-11-22', '2022-11-21', freq='D').week.values)


Вывод в json

{
      "date": "2022-01-02",
      "stock": 450,
      "green_estimate": 300,
      "yellow_estimate": 100,
      "red_estimate": 50
    },

Ниже пример

In [23]:
for i, row in df_total.iterrows():
    print(row['cum_sum_diff'])

-886
-594
-9
528
644
813
1384
2157
2993
2997
2290
1985
2025
2218
2638
1662
1802
1618
1307
1232
1050
890
792
480
351
197
-108
-571
-753
-612
-659
-828
-997
-962
-1120
-1386
-1453
-1573
-1475
-1719
-1889
-2078
-2503
-2601
-2354
-2399
-2436
-2309
-2130
-2482
-2972
-4191


In [24]:
import random

obj_json = {}

date_idx = 1
obj_arr = []
for i in range(0, 30):
    obj_one = {}
    obj_one['date'] = f"2022-01-{date_idx}"
    obj_one['stock'] = 450
    obj_one['green_estimate'] = 300 + random.randint(50, 100)
    obj_one['yellow_estimate'] = 100 + random.randint(20, 50)
    obj_one['red_estimate'] = 50 + random.randint(20, 35)
    obj_arr.append(obj_one)
    date_idx +=1
    
obj_json['message'] = obj_arr

with open("./example.json", "w") as json_file:
    json.dump(obj_json, json_file)